[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Scientific Programming Methods

## Discrete Optimization - Solving Sudoku Board

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 12/11/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0012LinearFitL1.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

import numba

# Machine Learning

# Optimization
import cvxpy as cp

# Image Processing / Computer Vision

# Miscellaneous
import math
from platform import python_version
import random
import time

# Typing
from typing import Callable, List, Optional, Tuple, Union

# Visualization
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
vallToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

# warnings.filterwarnings("ignore")

seedNum = 640 # 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme
# sns.set_palette("tab10")

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

BOARD_NUM_ROWS =  9
BOARD_NUM_COLS =  BOARD_NUM_ROWS


In [ ]:
# Course Packages


In [ ]:
# Auxiliary Functions

def DrawSudokuBoard( mB: np.ndarray, numRows: int, /, *, hA: Optional[plt.Axes] = None, figSize: Tuple[int, int] = FIG_SIZE_DEF ) -> plt.Axes:

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)
    else:
        hF = hA.get_figure()
    
    hRect = Rectangle(xy = (0, 0), width = numRows, height = numRows, edgecolor = 'k', fill = False, linewidth = 4)
    hA.add_patch(hRect)

    # Cell Borders
    for ii in range(1, numRows):
        hRect = Rectangle(xy = (ii, 0), width = 1, height = numRows, edgecolor = 'k', fill = False)
        hA.add_patch(hRect)
    
    for ii in range(1, numRows):
        hRect = Rectangle(xy = (0, ii), width = numRows, height = 1, edgecolor = 'k', fill = False)
        hA.add_patch(hRect)
    
    # Block Borders
    for ii in range(0, numRows, 3):
        hRect = Rectangle(xy = (ii, 0), width = 3, height = numRows, edgecolor = 'k', fill = False, linewidth = 2.5)
        hA.add_patch(hRect)
    
    # Block Borders
    for ii in range(0, numRows, 3):
        hRect = Rectangle(xy = (0, ii), width = numRows, height = 3, edgecolor = 'k', fill = False, linewidth = 2.5)
        hA.add_patch(hRect)
    
    # Fill in the data
    #
    # The rows of mD are of the form (i, j, k) where i is the row counting from
    # the top, j is the column, and k is the value. To place the entries in the
    # boxes, j is the horizontal distance, numRows + 1 - i is the vertical
    # distance. We subtract 0.5 to center the clue in the box.

    for ii in range(np.size(mB, 0)):
        hA.text(mB[ii, 1] - 0.5, (numRows + 0.45) - mB[ii, 0], f'{mB[ii, 2]:d}', fontsize = 30, horizontalalignment = 'center', verticalalignment = 'center')

    hA.set(xticks = [], yticks = [], xlim = (-0.5, numRows + 0.5), ylim = (-0.5, numRows + 0.5))
    
    return hA



In [ ]:
# Parameters

# Data
boardUrl = r'https://github.com/FixelAlgorithmsTeam/FixelCourses/raw/refs/heads/master/DataSets/SudokuBoard.txt'


## Solving Sudoku Board

![](https://i.postimg.cc/SxwWYmcC/png-transparent-slitherlink-jigsaw-puzzles-web-sudoku-others-angle-text-rectangle-thumbnail.png)
<!-- ![](https://i.imgur.com/w1d0Abb.png) -->

Solving a 9x9 Sudoku board using Linear Programming.  

The modeling is by a tensor of size `(9, 9, 9)` of a binary variable.


* <font color='brown'>(**#**)</font> The motivation for the regualrization can be interpreted in many ways: Bayesian Prior (Gaussian, Laplace, etc...), Model (Sparse, Shifted), Kernel, etc...



## Generate Data


The data generates both the train and the test data.

In [ ]:
# Generate / Load the Data

# mB[ii, jj, cellVal]: ii - Row, jj - Col, cellVal - Value
# Uses 1 based indexing
mB = np.loadtxt(boardUrl, dtype = np.uint8)


In [ ]:
# Display Data 

hF, hA = plt.subplots(figsize = (8, 8))
hA = DrawSudokuBoard(mB, BOARD_NUM_ROWS, hA = hA)
hA.set_title('The Sudoku Board');

# hA.legend();

* <font color='red'>(**?**)</font> Think of a model to solve the problem with _Integer Programming_.

## Model

Building a "Convex" Binary Linear Programming model.



In [ ]:
# The Model

numRows = BOARD_NUM_ROWS

# For Integer Programming we could just create a 2D array and impose
# constraints and values. 
# Yet to use Linear Programming we will use a binary formulation by setting
# a 3D tensor `tX` where `[9, 9, 9] = size(tX)` and if `tX(ii, jj, kk) = 1`
# it suggests that the value of the `ii, jj` cell on the board is `kk`.

numVar = numRows ** 3

# Impose Constraints
# While conceptually the data is 3D tensor, in practice we solve:
# arg min_x    f^t * x    (LP Objective)
# subject to   A * x = b  (Equality Constraint)
#             0 <= x <= 1
# 1. Each column      `sum(tX(:, jj, kk)) = 1`.
# 2. Each row         `sum(tX(ii, :, kk)) = 1`.
# 3. Each depth slice `sum(tX(ii, jj, :)) = 1`.
# 4. Each sub grid    `sum(tX(..., ..., kk)) = 1`.
# 5. For each given index `tX(ii, jj, clueVal) = 1`.  
#    We can also limit the lower value for those indices to 1.
# 6. Continuous binary variable `0 <= tX <= 1`.

vF = np.zeros(numVar)
numClues = np.size(mB, 0)
numConst = 4 * (numRows ** 2) #<! Equality to clues using lower bounds

# Constraint Matrix
# mA * vX = vB;
# Assuming `vX = np.ravel(tX, order = 'F')` -> Column based.
mA   = np.zeros(shape = (numConst, numVar))
conA = 0 #<! Index of the constraint


In [ ]:
# Columns Constraints
# Each column per slice of kk

itmIdx = 0 #<! First item in Column / Row / 3rd Dim Slice index
for ii in range(numRows * numRows):
    mA[conA, itmIdx:(itmIdx + numRows)] = 1 #<! Sum over a column (Contiguous)
    itmIdx += numRows #<! Move to the next column
    conA   += 1

* <font color='blue'>(**!**)</font> Optimize the above using [Kronecker Product](https://en.wikipedia.org/wiki/Kronecker_product).  
   See [`np.kron()`](https://numpy.org/doc/stable/reference/generated/numpy.kron.html).

In [ ]:
# Rows Constraints
# Each row per slice of kk

itmIdx = 0 #<! First item in Column / Row / 3rd Dim Slice index
for ii in range(numRows * numRows):
    # Python excludes the last index
    mA[conA, itmIdx:(itmIdx + (numRows * numRows)):numRows] = 1 #<! Sum over a row
    if (((itmIdx + 1) % numRows) == 0):
        # New slice of kk
        itmIdx = (itmIdx - numRows + 1) + (numRows * numRows)
    else:
        itmIdx += 1
    conA += 1

* <font color='blue'>(**!**)</font> Optimize the above using [Kronecker Product](https://en.wikipedia.org/wiki/Kronecker_product).  
   See [`np.kron()`](https://numpy.org/doc/stable/reference/generated/numpy.kron.html).

In [ ]:
# Depth Slice Constraints
# Each cell on the 3rd dimension slice

itmIdx = 0 #<! First item in Column / Row / 3rd Dim Slice index
for ii in range(numRows * numRows):
    # Python excludes the last index
    mA[conA, itmIdx:(itmIdx + numVar):(numRows * numRows)] = 1 #<! Sum over the 3rd dimension
    itmIdx += 1
    conA   += 1


In [ ]:
# Sub Grid Constraints
# Summing over 3x3 sub grid

itmIdx = 0; #<! First item in Column / Row / 3rd Dim Slice index
for kk in range(numRows):
    for nn in range(0, 9, 3):
        for mm in range(0, 9, 3):
            for jj in range(3):
                for ii in range(3):
                    jn = jj + nn
                    im = ii + mm
                    itmIdx = (kk * (numRows * numRows)) + (jn * numRows) + im
                    mA[conA, itmIdx] = 1
            conA += 1


In [ ]:
# Boundary Constraints
vB = np.ones(numConst)

vL = np.zeros(numVar) #<! Lower Bound - 0
vU = np.ones(numVar)  #<! Upper Bound - 1


In [ ]:
# Clues Constraints
# Set vL according to input data (Clues)
for ii in range(numClues):
    # `mB` is 1 based
    clueIdx = (mB[ii, 0] - 1) + ((mB[ii, 1] - 1) * numRows) + ((mB[ii, 2] - 1) * numRows * numRows)
    vL[clueIdx] = 1

## Solve the Linear Programming Problem

In [ ]:
# SciPy Solution

oRes = sp.optimize.linprog(np.zeros(numVar), A_eq = mA, b_eq = vB, bounds = np.column_stack((vL, vU)))

assert (oRes.success), 'The problem is not solved.'
print('Problem is solved.')


In [ ]:
# Extract the Solution Board

vX = oRes['x']
mS = np.argmax(np.reshape(vX, (numRows, numRows, numRows), order = 'F'), axis = 2) + 1

* <font color='red'>(**?**)</font> Explain the solution. Is it guaranteed to hold all constraints? Explain.
<!--
```python
# Check
print(np.sum(mS, axis = 0))
print(np.sum(mS, axis = 1))
print(np.sum(np.reshape(mS, (3, 3, 3, 3)), axis = (1, 3)))
```
-->


In [ ]:
# Draw the Board

vI = np.tile(range(1, numRows + 1), numRows) #<! Replicate the vector
vJ = np.repeat(range(1, numRows + 1), numRows) #<! Replicates the items

hF, hA = plt.subplots(figsize = (8, 8))
hA = DrawSudokuBoard(np.column_stack((vI, vJ, np.ravel(mS, order = 'F'))), BOARD_NUM_ROWS, hA = hA)
hA.set_title('The Sudoku Board - LP Solution');


## Solve the Integer Linear Programming

In [ ]:
# DCP Solution

#===========================Fill This===========================#
# 1. Formulate the problem in CVXPY.  
#    Use `vX` for the optimal argument.
# !! You may find `cp.Variable(<>, boolean = True)` useful.
# !! Pay attention the problem is a feasibility problem.

# Model Data
vX = cp.Variable(numVar, boolean = True) #<! Variable

# Model Problem
cpObjFun = cp.Minimize(0) #<! Objective Function
cpConst  = [mA @ vX == vB, vX <= vU, vX >= vL] #<! Constraints
oCvxPrb  = cp.Problem(cpObjFun, cpConst) #<! Problem

oCvxPrb.solve(solver = cp.SCIP)
#===============================================================#

vX = vX.value

assert (oCvxPrb.status == 'optimal'), 'The problem is not solved.'
print('Problem is solved.')


In [ ]:
# Extract the Solution Board

mS = np.argmax(np.reshape(vX, (numRows, numRows, numRows), order = 'F'), axis = 2) + 1

In [ ]:
# Draw the Board

vI = np.tile(range(1, numRows + 1), numRows) #<! Replicate the vector
vJ = np.repeat(range(1, numRows + 1), numRows) #<! Replicates the items

hF, hA = plt.subplots(figsize = (8, 8))
hA = DrawSudokuBoard(np.column_stack((vI, vJ, np.ravel(mS, order = 'F'))), BOARD_NUM_ROWS, hA = hA)
hA.set_title('The Sudoku Board - ILP Solution');


* <font color='red'>(**?**)</font> Explain the solution. Is it guaranteed to hold all constraints? Explain.